In [3]:
# generates indirect feature vectors for all fingerprint in the dataframe

import pandas as pd
from scipy.spatial import Delaunay
import functions as fn
import json

def generatefeatures(df):

    # fetch name of fingerprint for current index and return it
    def get_name(index):
        name = df['path'][index]
        name = name.replace('/', '_')
        name = name.replace('.tif', '')
        return name
    
    # fetch fingerprint for current index and return it
    def fetch_fingerprint(index):
        fingerprint = []
        fingerprint.append(pd.DataFrame(df['mv'][index], columns=['x', 'y', 'value']))
        return fingerprint

    # generate minutiae from fingerprint and return points and minutiae
    def generate_minutiae(fingerprint):
        points = []
        minutiae = []
        for i in range(len(fingerprint[0])):
            points.append([fingerprint[0]['x'][i], fingerprint[0]['y'][i]])
            minutiae.append([fingerprint[0]['x'][i], fingerprint[0]['y'][i], fingerprint[0]['value'][i]])
        return points, minutiae

    # perform delaunay triangulation and return triangles
    def triangulate(points, minutiae):
        tri = Delaunay(points)

        triangles = []
        for i in range(len(tri.simplices)):
            triangles.append([minutiae[tri.simplices[i][0]], minutiae[tri.simplices[i][1]], minutiae[tri.simplices[i][2]]])
        return triangles
    
    # store entire database into a single array
    db_array = []

    # generate indirect features for all triangles
    
    name = get_name(0)
    fingerprint = fetch_fingerprint(0)
    points, minutiae = generate_minutiae(fingerprint)
    triangles = triangulate(points, minutiae)
    features_array = fn.ret_arr(triangles)
    db_array.append([name, features_array])

    return points
    

In [4]:
json_files = [r'Datasets\anguli_10_100_fingernet.json']
df = pd.read_json(json_files[0],orient='records')
fea = generatefeatures(df)
fea

[[139.0, 379.0],
 [106.0, 388.0],
 [309.0, 324.0],
 [105.0, 362.0],
 [74.0, 390.0],
 [226.0, 371.0],
 [317.0, 282.0],
 [235.0, 340.0],
 [293.0, 173.0],
 [154.0, 293.0],
 [157.0, 165.0],
 [253.0, 42.0],
 [90.0, 293.0],
 [422.0, 325.0],
 [228.0, 237.0],
 [195.0, 379.0],
 [299.0, 374.0],
 [201.0, 291.0],
 [165.0, 221.0],
 [123.0, 227.0],
 [283.0, 414.0],
 [177.0, 324.0],
 [149.0, 253.0],
 [140.0, 462.0],
 [362.0, 322.0],
 [146.0, 362.0],
 [82.0, 132.0],
 [242.0, 410.0],
 [180.0, 363.0],
 [365.0, 188.0],
 [210.0, 146.0],
 [370.0, 93.0],
 [333.0, 358.0],
 [331.0, 130.0],
 [100.0, 187.0],
 [357.0, 213.0],
 [348.0, 419.0],
 [297.0, 177.0],
 [450.0, 372.0],
 [155.0, 429.0],
 [162.0, 340.0],
 [61.0, 250.0],
 [283.0, 189.0],
 [365.0, 171.0],
 [220.0, 75.0],
 [197.0, 33.0],
 [325.0, 436.0],
 [420.0, 434.0],
 [163.0, 355.0],
 [196.0, 83.0]]

In [5]:
import numpy as np

def two_nearest_neighbors(points):
    """
    Constructs a Delaunay triangulation of a set of points using the two nearest neighbor algorithm.

    Args:
        points (np.ndarray): An array of points, where each point is a 2D vector.

    Returns:
        np.ndarray: An array of triangles, where each triangle is a 3D vector of indices into the `points` array.
    """
    triangulation = []

    for i in range(len(points)):
        nearest_neighbors = np.argpartition(np.linalg.norm(points[i] - points, axis=1), 2)[:2]

        for j in nearest_neighbors:
            if j != i:
                for k in nearest_neighbors:
                    if k != i and k != j and not point_in_triangle(points[i], points[j], points[k]):
                        triangulation.append([i, j, k])

    return np.array(triangulation)

def point_in_triangle(p, a, b, c):
    """
    Determines whether a point is inside a triangle.

    Args:
        p (np.ndarray): The point to be tested.
        a (np.ndarray): The first vertex of the triangle.
        b (np.ndarray): The second vertex of the triangle.
        c (np.ndarray): The third vertex of the triangle.

    Returns:
        bool: True if the point is inside the triangle, False otherwise.
    """
    area = ((b[1] - c[1]) * (p[0] - c[0]) - (b[0] - c[0]) * (p[1] - c[1]))
    if area == 0:
        return False

    sign = area / abs(area)
    return ((b[1] - c[1]) * (a[0] - c[0]) - (b[0] - c[0]) * (a[1] - c[1])) * sign > 0 and ((c[1] - a[1]) * (p[0] - a[0]) - (c[0] - a[0]) * (p[1] - a[1])) * sign > 0

points = np.array([[0, 0], [1, 0], [0, 1]])
triangulation = two_nearest_neighbors(points)
print(triangulation)


[]


In [6]:
triangulation

array([], dtype=float64)